In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from tqdm import tqdm

# Load the dataset
df = pd.read_csv('final_manglish.csv')

# Convert Sentiment Classes to Integers
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2, 'Mixed Feelings': 3, 'Not_relevant': 4}
df['Sentiment_Class'] = df['Sentiment_Class'].map(label_mapping)

# Check and preprocess the 'commentText' column
comments = df['commentText'].astype(str).tolist()

# Load pre-trained DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=len(label_mapping))

# Tokenize and encode the dataset
encoded_data = tokenizer.batch_encode_plus(
    comments,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    return_tensors='pt',
    truncation=True
)

# Extract input tensors
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']
labels = torch.tensor(df['Sentiment_Class'].values)

# Split the dataset
train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = train_test_split(
    input_ids, labels, attention_mask, random_state=42, test_size=0.2
)

# Create DataLoader for train and test sets
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * 2
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Train the model with early stopping based on validation loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 2
best_val_loss = float('inf')
patience = 0
for epoch in range(epochs):
    # Training loop with progress bar
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update progress bar
        progress_bar.set_postfix({'training_loss': f'{loss.item():.3f}'})

    # Validation
    model.eval()
    val_losses = []
    for batch in test_dataloader:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks, labels=labels)
            val_loss = outputs.loss.item()
            val_losses.append(val_loss)

    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f'Epoch {epoch + 1}/{epochs}, Test Loss: {avg_val_loss:.4f}')

    # Early stopping based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience = 0
    else:
        patience += 1
        if patience > 2:
            print("Early stopping triggered, no improvement in test loss.")
            break

# Evaluate the model on the test set
model.eval()
test_predictions = []
test_labels_flattened = test_labels.flatten().long()
with torch.no_grad():
    for batch in test_dataloader:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
        outputs = model(inputs, attention_mask=masks)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().numpy())

# Print classification report
print("Classification Report:")
print(classification_report(test_labels_flattened.cpu().numpy(), test_predictions, target_names=label_mapping.keys()))

# Save the model using torch.save()
output_dir = './distilbert_sentiment_model'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'tokenizer': tokenizer,
    'label_mapping': label_mapping
}, output_dir)

print("Model saved successfully at:", output_dir)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/2, Test Loss: 0.7953


Epoch 2/2, Test Loss: 0.7372
Classification Report:
                precision    recall  f1-score   support

      Negative       0.52      0.61      0.56       502
       Neutral       0.83      0.85      0.84      1295
      Positive       0.62      0.71      0.66       570
Mixed Feelings       0.60      0.03      0.05       113
  Not_relevant       0.76      0.71      0.73      1279

      accuracy                           0.72      3759
     macro avg       0.67      0.58      0.57      3759
  weighted avg       0.73      0.72      0.71      3759

Model saved successfully at: ./distilbert_sentiment_model


In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import pandas as pd
from tqdm import tqdm

# Load the dataset
df = pd.read_csv('final_manglish.csv')

# Convert Sentiment Classes to Integers
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2, 'Mixed Feelings': 3, 'Not_relevant': 4}
df['Sentiment_Class'] = df['Sentiment_Class'].map(label_mapping)

# Check and preprocess the 'commentText' column
comments = df['commentText'].astype(str).tolist()

# Load pre-trained DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=len(label_mapping))

# Tokenize and encode the dataset
encoded_data = tokenizer.batch_encode_plus(
    comments,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    return_tensors='pt',
    truncation=True
)

# Extract input tensors
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']
labels = torch.tensor(df['Sentiment_Class'].values)

# Concatenate input_ids and attention_mask tensors
combined_inputs = torch.cat((input_ids, attention_mask), dim=1)

# Oversampling using SMOTE
smote = SMOTE(random_state=42)
combined_inputs_resampled, labels_resampled = smote.fit_resample(combined_inputs, labels)

# Convert resampled tensors to PyTorch tensors
input_ids_resampled = torch.tensor(combined_inputs_resampled[:, :input_ids.shape[1]])
attention_mask_resampled = torch.tensor(combined_inputs_resampled[:, input_ids.shape[1]:])
labels_resampled = torch.tensor(labels_resampled)

# Split the resampled dataset
train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = train_test_split(
    input_ids_resampled, labels_resampled, attention_mask_resampled, random_state=42, test_size=0.2
)

# Create DataLoader for train and test sets
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * 2
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Train the model with early stopping based on validation loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 2
best_val_loss = float('inf')
patience = 0
for epoch in range(epochs):
    # Training loop with progress bar
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update progress bar
        progress_bar.set_postfix({'training_loss': f'{loss.item():.3f}'})

    # Validation
    model.eval()
    val_losses = []
    for batch in test_dataloader:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks, labels=labels)
            val_loss = outputs.loss.item()
            val_losses.append(val_loss)

    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f'Epoch {epoch + 1}/{epochs}, Test Loss: {avg_val_loss:.4f}')

    # Early stopping based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience = 0
    else:
        patience += 1
        if patience > 2:
            print("Early stopping triggered, no improvement in test loss.")
            break

# Evaluate the model on the test set
model.eval()
test_predictions = []
test_labels_flattened = test_labels.flatten().long()
with torch.no_grad():
    for batch in test_dataloader:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
        outputs = model(inputs, attention_mask=masks)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().numpy())

# Print classification report
print("Classification Report:")
print(classification_report(test_labels_flattened.cpu().numpy(), test_predictions, target_names=label_mapping.keys()))

# Save the model using torch.save()
output_dir = './distilbert_sentiment_model'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'tokenizer': tokenizer,
    'label_mapping': label_mapping
}, output_dir)

print("Model saved successfully at:", output_dir)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/2, Test Loss: 0.9051


Epoch 2/2, Test Loss: 0.8768
Classification Report:
                precision    recall  f1-score   support

      Negative       0.55      0.29      0.38      1251
       Neutral       0.83      0.85      0.84      1274
      Positive       0.55      0.47      0.51      1273
Mixed Feelings       0.49      0.77      0.60      1291
  Not_relevant       0.76      0.75      0.76      1296

      accuracy                           0.63      6385
     macro avg       0.64      0.63      0.62      6385
  weighted avg       0.64      0.63      0.62      6385

Model saved successfully at: ./distilbert_sentiment_model


In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('final_manglish.csv')

# Convert Sentiment Classes to Integers
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2, 'Mixed Feelings': 3, 'Not_relevant': 4}
df['Sentiment_Class'] = df['Sentiment_Class'].map(label_mapping)

# Upsample minority classes to match the size of the majority class
data_upsampled = pd.concat([
    df[df['Sentiment_Class'] == label_mapping['Positive']].sample(n=len(df[df['Sentiment_Class'] == label_mapping['Not_relevant']]), replace=True),
    df[df['Sentiment_Class'] == label_mapping['Negative']].sample(n=len(df[df['Sentiment_Class'] == label_mapping['Not_relevant']]), replace=True),
    df[df['Sentiment_Class'] == label_mapping['Not_relevant']],
    df[df['Sentiment_Class'] == label_mapping['Mixed Feelings']].sample(n=len(df[df['Sentiment_Class'] == label_mapping['Not_relevant']]), replace=True),
    df[df['Sentiment_Class'] == label_mapping['Neutral']].sample(n=len(df[df['Sentiment_Class'] == label_mapping['Not_relevant']]), replace=True)
])

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
encoded_data_upsampled = tokenizer(data_upsampled['commentText'].tolist(), padding=True, truncation=True, max_length=256, return_tensors='pt')

# Prepare target variable
labels_upsampled = torch.tensor(data_upsampled['Sentiment_Class'].values)

# Split the upsampled dataset
train_indices, test_indices = train_test_split(
    np.arange(len(encoded_data_upsampled.input_ids)),
    random_state=42,
    test_size=0.2
)

train_inputs = encoded_data_upsampled.input_ids[train_indices]
test_inputs = encoded_data_upsampled.input_ids[test_indices]
train_masks = encoded_data_upsampled.attention_mask[train_indices]
test_masks = encoded_data_upsampled.attention_mask[test_indices]
train_labels = labels_upsampled[train_indices]
test_labels = labels_upsampled[test_indices]

# Create DataLoader for train and test sets
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

# Load pre-trained DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=len(label_mapping))

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * 2
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 2
best_val_loss = float('inf')
patience = 0
for epoch in range(epochs):
    # Training loop
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update progress bar
        progress_bar.set_postfix({'training_loss': f'{loss.item():.3f}'})

    # Validation
    model.eval()
    val_losses = []
    for batch in test_dataloader:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks, labels=labels)
            val_loss = outputs.loss.item()
            val_losses.append(val_loss)

    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f'Epoch {epoch + 1}/{epochs}, Test Loss: {avg_val_loss:.4f}')

    # Early stopping based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience = 0
    else:
        patience += 1
        if patience > 2:
            print("Early stopping triggered, no improvement in test loss.")
            break

# Evaluate the model on the test set
model.eval()
test_predictions = []
test_labels_flattened = test_labels.flatten().long()
with torch.no_grad():
    for batch in test_dataloader:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
        outputs = model(inputs, attention_mask=masks)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().numpy())

# Print classification report
print("Classification Report:")
print(classification_report(test_labels_flattened.cpu().numpy(), test_predictions, target_names=label_mapping.keys()))

# Save the model
output_dir = './distilbert_sentiment_model'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'tokenizer': tokenizer,
    'label_mapping': label_mapping
}, output_dir)

print("Model saved successfully at:", output_dir)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 